# TP1: FEATURE EXTRACTION & VISUALIZATION

---



In this notebook, you will learn how to **extract radiomics features** and **visualize the important characteristics of the dataset**. We follow a typical data analysis pipeline.

<img src='https://raw.githubusercontent.com/RRouhi/EPU-IA-2022/main/images/pipeline.png'/>

First, download the necessary materials for the practical sessions.

In [ ]:
!git clone https://github.com/marsvn/PythonM2-jour3.git
%cd PythonM2-jour3/

### Installation of dependencies

In [ ]:
!pip install pyradiomics SimpleITK

## 1. Loading data

The dataset we will work on is a subset of the BraTS dataset (https://www.med.upenn.edu/cbica/brats2019/data.html). There are 10 patients in each of the Glioblastoma (`./data/preprocessed/GBM`) and Low-Grade Glioma (`./data/preprocessed/LGG`) classes. Each patient has five files: four MRI volumes of the brain under T1, T1-Gd, T2, and T2 Flair imaging modalities; and a tumour segmentation file. Let us start by considering **a single patient**:

In [ ]:
import os

root_dir = './data/preprocessed'

patient = 'GBM/TCGA-02-0037'
patient_dir = os.path.join(root_dir, patient)
patient_files = os.listdir(patient_dir)

print(patient_files)

We will now visualise the files of this patient. In the following, the data is returned in the `sitk` format. `SimpleITK` is a python library for registration and segmentation of biomedical images. Two test cases will be loaded and and saved.

In [ ]:
import glob
import matplotlib.pyplot as plt
import SimpleITK as sitk

def load_nii_volume_as_array(filename, with_header=False):
    """
    load NIfTI image into numpy array with axis order of [z,y,x]
    The output array shape is like [Depth, Height, Width]
    :param filename: the input file name, should be *.nii or *.nii.gz
    :param with_header: return header information
    :return: a numpy data array or numpy data array with header if set to True
    """
    img = sitk.ReadImage(filename)
    data = sitk.GetArrayFromImage(img)

    if with_header:
        origin, spacing, direction = img.GetOrigin(), img.GetSpacing(), img.GetDirection()
        return data, (origin, spacing, direction)
    else:
        return data

volume = load_nii_volume_as_array(os.path.join(patient_dir, patient_files[0]))
print(volume.shape)

There are $155$ slices in the volume, where each slice is $240\times240$px image. Let's visualise some of them.

In [ ]:
fig, axes = plt.subplots(ncols=5, nrows=4, figsize=(15, 10))
for i, img in enumerate(volume[::8]):
    axes.ravel()[i].set_title('Slice: %d' % (8 * i))
    axes.ravel()[i].imshow(img, cmap='Greys_r')

plt.tight_layout()

### Exercise 1

Visualize a single slice for each imaging modality and the segmentation mask

In [ ]:
fig, axes = plt.subplots(ncols=5, figsize=(10, 10))

for i, patient_file in enumerate(patient_files):
    volume = load_nii_volume_as_array("""CompleteHere""")
    axes[i].imshow(volume[80], cmap='Greys_r')
    axes[i].set_title(patient_file.split('_')[-1].split('.')[0])

plt.tight_layout()

## 2. Normalising data

To stabilise the downstream feature extraction and analysis, we first normalise the data. For input image $X$, we create normalised image $Z$ by normalising each pixel $x_{ij}$ according to,

$$z_{ij} = \frac{x_{ij} - \mu_X}{\sigma_X}.$$

Because of the abundance of background values in each volume, we compute $\mu_X$ and $\sigma_X$ only over the pixels of $X$ above a certain threshold.

In [ ]:
from glob import glob


def set_header_information(array, header):
    """
    Function to set header information to an array.
    :param array: array to set the header
    :param header: header information will be a tuple with (origin, spacing, direction)
    :return: an image in sitk format
    """
    img = sitk.GetImageFromArray(array)
    img.SetOrigin(header[0])
    img.SetSpacing(header[1])
    img.SetDirection(header[2])
    return img

def zscore_normalize(patient, image_type):
    """
    Function to Z-Score normalize an image from an image filepath. It will normalize the target
    image by subtracting the mean of the whole brain and dividing by the standard deviation.
    :param img_path: target MR brain image path
    :param mask: brain mask path for img
    :return: Normalized image in sitk format
    """

    img_path = glob(os.path.join(root_dir, patient, f'*{image_type}.nii.gz'))[0]
    img_data, header = load_nii_volume_as_array(img_path, with_header=True)
    mask = (img_data > img_data.mean()) == 1  # force the mask to be logical type

    mean = img_data[mask].mean()
    std = img_data[mask].std()

    normalized_data = (img_data - mean) / std
    normalized = set_header_information(normalized_data, header)

    return normalized

normalised_image = zscore_normalize(patient, 't1')
normalised_volume = sitk.GetArrayFromImage(normalised_image)

fig, axes = plt.subplots(ncols=2, figsize=(10, 3))

def plot_histogram(ax, volume, title):
    ax.hist(volume.flatten(), color='gray')
    ax.set_xlabel('intensity')
    ax.set_yscale('log')
    ax.set_title(title)

plot_histogram(axes[0], volume, 'unnormalised')
plot_histogram(axes[1], normalised_volume, 'normalised')

### Extract mask information

If we visualise a given slice of any mask volume, we see that it is in fact a multi-class image, where different values indicate different tumour regions: enhancing tumour (ET), peritumoral edema (ED), necrosis.

In [ ]:
mask_path = glob(os.path.join(root_dir, patient, '*seg*'))[0]
mask, header = load_nii_volume_as_array(mask_path, with_header=True)

plt.imshow(mask[96])

From this multi-class mask, we can derive the regions as separate binary masks. We define a special purpose function to extract each mask region as a separate binary mask.

In [ ]:
def get_masks(patient):
    """
    Function to read one subject's mask from the general input directory specified by patient ID
    :param patient: the patient ID
    :return: a dict for the 'seg' with key as type of seg and value as nD array. For image a nD array
    """

    mask_path = glob(os.path.join(root_dir, patient, '*seg*'))[0]
    mask, header = load_nii_volume_as_array(mask_path, with_header=True)

    label_full = mask.copy()
    label_full[mask != 0] = 1        # take all regions

    label_nec = mask.copy()
    label_nec[mask != 1] = 0         # necrosis only

    label_core = mask.copy()
    label_core[mask == 2] = 0        # no edema
    label_core[label_core != 0] = 1  # keep necrosis, ET, NET i.e. tumor core

    label_et = mask.copy()
    label_et[mask != 4] = 0          # ET only
    label_et[mask == 4] = 1

    all_masks = {
        'label_full' : set_header_information(label_full, header),
        'label_nec' : set_header_information(label_nec, header),
        'label_core' : set_header_information(label_core, header),
        'label_et' : set_header_information(label_et, header)
    }

    return all_masks

all_masks = get_masks(patient)

fig, axes = plt.subplots(ncols=4, figsize=(15, 5))

for ax, mask_key, mask_volume in zip(axes, all_masks.keys(), all_masks.values()):
    ax.set_title(mask_key)
    ax.imshow(sitk.GetArrayFromImage(mask_volume)[96], cmap='Greys_r')

**Exercise:** Replace the expression `CompleteHere` in the following script to:
- load images and segmentations from the two patients
- apply normalisation to MRI images
- save the normalised images

In [ ]:
#@title Exercise 2
"""
>>> Preprocess patient 1
"""

patient1 = 'GBM/TCGA-02-0037/'

# normalise volumes
patient1_t1 = zscore_normalize(patient1, 't1')
patient1_t1ce = zscore_normalize(patient1, 't1ce')
patient1_t2 = zscore_normalize(patient1, 't2')
patient1_flair = zscore_normalize(patient1, 'flair')

# Write normalised volumes
sitk.WriteImage("""CompleteHere""", 'patient1_t1_normalize.nii.gz')
sitk.WriteImage("""CompleteHere""", 'patient1_t1ce_normalize.nii.gz')
sitk.WriteImage("""CompleteHere""", 'patient1_t2_normalize.nii.gz')
sitk.WriteImage("""CompleteHere""", 'patient1_flair_normalize.nii.gz')

# Derive masks
patient1_masks = get_masks(patient1)

# Write masks
sitk.WriteImage(patient1_masks['label_full'], 'patient1_seg_label_full.nii.gz')
sitk.WriteImage(patient1_masks['label_nec'], 'patient1_seg_label_nec.nii.gz')
sitk.WriteImage(patient1_masks['label_core'], 'patient1_seg_label_core.nii.gz')
sitk.WriteImage(patient1_masks['label_et'], 'patient1_seg_label_et.nii.gz')

"""
>>> Preprocess patient 2
"""

patient2 = 'LGG/TCGA-CS-5396/'

# Normalise volumes
patient2_t1 = zscore_normalize(patient2, 't1')
patient2_t1ce = zscore_normalize(patient2, 't1ce')
patient2_t2 = zscore_normalize(patient2, 't2')
patient2_flair = zscore_normalize(patient2, 'flair')

# Write normalised volumes
sitk.WriteImage("""CompleteHere""", 'patient2_t1_normalize.nii.gz')
sitk.WriteImage("""CompleteHere""", 'patient2_t1ce_normalize.nii.gz')
sitk.WriteImage("""CompleteHere""", 'patient2_t2_normalize.nii.gz')
sitk.WriteImage("""CompleteHere""", 'patient2_flair_normalize.nii.gz')

# Derive masks
patient2_masks = get_masks(patient2)

# Write masks
sitk.WriteImage("""CompleteHere""", 'patient2_seg_label_full.nii.gz')
sitk.WriteImage("""CompleteHere""", 'patient2_seg_label_nec.nii.gz')
sitk.WriteImage("""CompleteHere""", 'patient2_seg_label_core.nii.gz')
sitk.WriteImage("""CompleteHere""", 'patient2_seg_label_et.nii.gz')

## 3. Feature extraction

Now that we have prepared our image inputs, we need to define the parameters and instantiate the extractor. We use the default settings of the extractor. Let us extract features for patient 1, under the t1 modality, for the full tumour region.

In [ ]:
from radiomics import featureextractor  # This module is used for interaction with pyradiomics

# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor()

imagePath = 'patient1_t1_normalize.nii.gz'
maskPath = 'patient1_seg_label_full.nii.gz'

result = extractor.execute(imagePath, maskPath)
print(result.values())

As you can see, Pyradiomics returns an ordered dictionary that is visually not very pleasant. We can instead turn this result into a `pandas` format. The keys in the dictionary will be used as the index (labels for the rows), with the values of the features as the values in the rows.

In [ ]:
import pandas as pd

df = pd.Series(result)
print(df)

### Process all

Now we will use a loop to extract a set of features for our two patients with each of their segmentations. The idea is to loop over each image the associated segmentations. We construct a `pandas.DataFrame` object to list each combination.

In [ ]:
image_names = sorted(glob('patient1*normalize*'))
mask_names = sorted(glob('patient1_seg*'))

index = pd.MultiIndex.from_product([image_names, mask_names], names = ['Image', 'Segmentation'])
df_patient1 = pd.DataFrame(index=index).reset_index()

image_names = sorted(glob('patient2*normalize*'))
mask_names = sorted(glob('patient2_seg*'))

index = pd.MultiIndex.from_product([image_names, mask_names], names = ['Image', 'Segmentation'])
df_patient2 = pd.DataFrame(index=index).reset_index()

database_df = pd.concat([df_patient1, df_patient2]).reset_index(drop=True)
database_df

Now we can iterate over each row of our `pandas.DataFrame` and extract features for each combination of modality and tumour interest region.

In [ ]:
from radiomics import featureextractor

paramPath = os.path.join('exampleSettings', 'Params.yaml')
extractor = featureextractor.RadiomicsFeatureExtractor(paramPath)

results_extraction = pd.DataFrame()
for row_idx, row in database_df.iterrows():
    print('process:'  '   ----   ' 'Image: ', row['Image'], ' Segmentation: ', row['Segmentation'])
    result_extraction = pd.Series(extractor.execute(row['Image'], row['Segmentation']))
    feature_vector = pd.concat([pd.Series(row), result_extraction])
    feature_vector.name = row_idx
    results_extraction = results_extraction.join(feature_vector, how='outer')

# Transpose DataFrame to have one row per analysis
results_extraction = results_extraction.T

The result consists of a table where each row corresponds to an modality/interest region pair, and each column is a radiomics feature. We can print a few rows using `pd.DataFrame.head()`:

In [ ]:
results_extraction.head()

### Compare feature signatures

Now, let's compare the feature "signatures" for our two patients. Because we would like to plot the feature vectors, we will create `numpy` arrays for those features with prefix `original_` (i.e. excluding meta-features). We begin by extracting the relevant parts of `results_extraction` above.

In [ ]:
# Extract patient and modality identifiers
patient = results_extraction['Image'].str.split('_').str[0]
sequence = results_extraction['Image'].str.split('_').str[1]

# Extract segmentation type
segtype = results_extraction['Segmentation'].str.split('_').str[-1].str.split('.').str[0]

# Extract non-meta features
feature_columns = [col for col in results_extraction if col.startswith('original')]

dataframe_cleaned = results_extraction[feature_columns]
dataframe_cleaned.insert(0, 'patient', patient)
dataframe_cleaned.insert(1, 'sequence', sequence)
dataframe_cleaned.insert(2, 'segmentation', segtype)

dataframe_cleaned.head()

**Exercise:** Complete the following script to:
- extract the feature signatures for a given modality/region combination by slicing `dataframe_cleaned`:
- plot the two feature vectors and their difference.

Notes:
- Feature values have a wide range of magnitudes and will be plotted on a log scale. You can have fun and change the input modality or the segmentation type.


In [ ]:
#@title Exercise 3
modality = '''CompleteHere'''
seg = '''CompleteHere'''
features_p1 =  dataframe_cleaned.iloc['''CompleteHere'''].to_numpy()
features_p2 =  dataframe_cleaned.iloc['''CompleteHere'''].to_numpy()


import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 1, sharex=True, squeeze=True, figsize=(20, 10))

axes[0].plot('''CompleteHere''', label=f"features from patient 1 for the segmentation {seg} in modality {modality}", color='red')
axes[0].plot('''CompleteHere''', label=f"features from patient 2 for the segmentation {seg} in modality {modality}", color='green')
axes[0].set_yscale('symlog')
axes[0].set_ylabel('symlog')
axes[0].set_xlabel('feature id')
axes[0].set_xticklabels(feature_columns, rotation='vertical')
axes[0].legend()

axes[1].plot('''CompleteHere''' - features_p2_t1_full)
axes[1].set_yscale('symlog')
axes[1].set_ylabel('symlog')
axes[1].set_xlabel('feature id')
axes[1].set_xticks(range(len(features_p1_t1_full)))
axes[1].set_xticklabels('''CompleteHere''', rotation=90)
axes[1].set_title('''CompleteHere''')
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

## 4. Exploring and Visualizing Data

The first step in applying machine learning is to understand the data, so as to be able to identify a good target to learn, as well as an appropriate learning algorithm among many alternatives.

The analysis above has been applied on two patients only. Since our time in this practical session is limited, the features on the entire BraTS dataset has been extracted into a csv file located at `./data/radiomics_analysis_cleaned.csv`. This file contains one row per patient per input modality (T1, T1ce, T2, flair) per type of tumor ROI ($4\times4=16$ rows per patient). With the feaures from the full cohort, we can begin to explore the data.

In [ ]:
import pandas as pd
from IPython.display import display

path_dataset = './data/radiomics_analysis_cleaned.csv'

data = pd.read_csv(path_dataset)
data.head(10)

We can generate a full `pandas.DataFrame` which represents all the features for a patient. This gives $16\times100 = 1600$ featues per patient.

In [ ]:
full_features_df = data.pivot_table(index=['patient', 'label'],
                                    columns=['sequence', 'segmentation'],
                                    values=data.columns[4:])
full_features_df.columns = ['_'.join(col).strip() for col in full_features_df.columns.values]
full_features_df.reset_index(level=1, inplace=True)

label_counts = full_features_df['label'].value_counts()

print('Patients per disease class: %d HGG, %d LGG' % (label_counts.HGG, label_counts.LGG))
print('Numbers of features: %d' % (len(full_features_df.columns) -1))

### Pairwise joint distributions

We can see the join distribution of any pair of columns/attributes/variables/features by using the pairplot function offered by `Seaborn`, which is a plotting library based on `Matplotlib`. We do this for the first 5 features. Each datapoint is a patient and the colour indicates the disease class.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# filter one sequence type and a segmentation type (you can change the sequence type and segmentation type)
d = data[(data['sequence']=='t1ce')  & (data['segmentation']=='full')].iloc[:, 3:9]

sns.pairplot(d, hue='label', height=2.5)
plt.tight_layout()
plt.show()

#### Interpretation

We can observe in the plots that some pairs of features yield a high degree of discrimination between LGG (in orange) and HGG (in blue) patients.
For instance, the plot in the second row, first column, between the features 10Percentile (10th percentile of image intensities) and 90Percentile (idem with 90th) appears to cluster each cancer subtypes into roughly distinct clusters.

In these 2D feature subspaces, a simple line separating the two classes would form the basis of a decision system for classifying an MRI. Multi-dimensional models, applied on many hundreds of features, essentially perform the same method, by fitting a *hyperplane* such to best separate the datapoints from each class.

### Plot features' dependency as a heatmap

Plotting all pairwise distributions may still be hard to interpret when we have a lot of variables in the dataset. Instead, we can just plot the correlation matrix to quantify the linear relationship between variables.

A heat map uses colour to depict the correlation of each feature with every other feature. A heatmap may furthremore be clustered (unsupervised machine learning) to show groupings of similar features.

In [ ]:
import pandas as pd
import seaborn as sns

# filter one sequence type and a segmentation type (you can change the sequence type and segmentation type)
d = data[(data['sequence']=='t1ce')  & (data['segmentation']=='full')].iloc[:, 3:] # the full feature one is too big

# create the correlation matrix
corr = d.select_dtypes(include=['number']).corr()

# Set up the matplotlib figure, make it big!
f, ax = plt.subplots(figsize=(15, 10))

# Draw the heatmap using seaborn
sns.heatmap(corr, vmax=.8, square=True)
plt.show()

### Cluster the heatmap

Though useful, heatmaps tell a much better story if the features are clustered. Here we will take a smaller subset of the features and cluster them. In this dendrogram, there are 2 major groups, and many smaller groupings.

In [ ]:
# Choose a subset of features for clustering
dd = d.iloc[:,3:50]

pp = sns.clustermap(dd.corr(), linewidths=.5, figsize=(13,13))
_ = plt.setp(pp.ax_heatmap.get_yticklabels(), rotation=0)

plt.show()